# Double Texting

Seamless handling of [double texting](https://langchain-ai.github.io/langgraph/concepts/double_texting/) is important for handling real-world usage scenarios, especially in chat applications.

Users can send multiple messages in a row before the prior run(s) complete, and we want to ensure that we handle this gracefully.

## Reject

A simple approach is to [reject](https://langchain-ai.github.io/langgraph/cloud/how-tos/reject_concurrent/) any new runs until the current run completes.

In [1]:
%%capture --no-stderr
%pip install -U langgraph_sdk

In [1]:
from langgraph_sdk import get_client
url_for_cli_deployment = "http://localhost:8123"
client = get_client(url=url_for_cli_deployment)

In [2]:
import httpx
from langchain_core.messages import HumanMessage

# Create a thread
thread = await client.threads.create()

# Create to dos
user_input_1 = "Add a ToDo to follow-up with DI Repairs."
user_input_2 = "Add a ToDo to mount dresser to the wall."
config = {"configurable": {"user_id": "Lance"}}
graph_name = "task_maistro" 

run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_1)]}, 
    config=config,
)
try:
    await client.runs.create(
        thread["thread_id"],
        graph_name,
        input={"messages": [HumanMessage(content=user_input_2)]}, 
        config=config,
        multitask_strategy="reject",
    )
except httpx.HTTPStatusError as e:
    print("Failed to start concurrent run", e)

Failed to start concurrent run Client error '409 Conflict' for url 'http://localhost:8123/threads/7d3cad29-105e-42dc-984e-54e8b7de9c1d/runs'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/409


In [3]:
from langchain_core.messages import convert_to_messages

# Wait until the original run completes
await client.runs.join(thread["thread_id"], run["run_id"])

# Get the state of the thread
state = await client.threads.get_state(thread["thread_id"])
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

Add a ToDo to follow-up with DI Repairs.
================================== Ai Message ==================================

It looks like you already have a task to follow-up with DI Repairs on your ToDo list. Would you like to update the status or details of that task instead?


## Enqueue

We can use [enqueue](https://langchain-ai.github.io/langgraph/cloud/how-tos/enqueue_concurrent/https://langchain-ai.github.io/langgraph/cloud/how-tos/enqueue_concurrent/) any new runs until the current run completes.

In [4]:
# Create a new thread
thread = await client.threads.create()

# Create new ToDos
user_input_1 = "Send Erik his t-shirt gift this weekend."
user_input_2 = "Get cash and pay nanny for 2 weeks. Do this by Friday."
config = {"configurable": {"user_id": "Lance"}}
graph_name = "task_maistro" 

first_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_1)]}, 
    config=config,
)

second_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_2)]}, 
    config=config,
    multitask_strategy="enqueue",
)

# Wait until the second run completes
await client.runs.join(thread["thread_id"], second_run["run_id"])

# Get the state of the thread
state = await client.threads.get_state(thread["thread_id"])
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

Send Erik his t-shirt gift this weekend.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_qXDVcN1zjyQUj4aRNlQ3BQOq)
 Call ID: call_qXDVcN1zjyQUj4aRNlQ3BQOq
  Args:
    update_type: todo
================================= Tool Message =================================

Document 45114b64-a6b7-4d24-bbad-b339da309224 updated:
Plan: Update the task to send Erik his t-shirt gift this weekend, and set the status to 'not started'.
Added content: Send Erik his t-shirt gift this weekend.

New ToDo created:
Content: {'task': 'Send Erik his t-shirt gift this weekend.', 'time_to_complete': 15, 'deadline': '2024-01-06T00:00:00Z', 'status': 'not started'}
================================== Ai Message ==================================

I've added the task to send Erik his t-shirt gift this weekend to your ToDo list. If you need any help with it, just let me

## Interrupt

We can use [interrupt](https://langchain-ai.github.io/langgraph/cloud/how-tos/interrupt_concurrent/) to interrupt the current run, but save all the work that has been done so far up to that point.


In [5]:
# Create a new thread
thread = await client.threads.create()

# Create new ToDos
user_input_1 = "Order turkey for Thanksgiving by Friday."
user_input_2 = "Never mind, Thanksgiving is the 28th! Order Ham for Thanksgiving by next Friday."
config = {"configurable": {"user_id": "Lance"}}
graph_name = "task_maistro" 

interrupted_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_1)]}, 
    config=config,
)

second_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_2)]}, 
    config=config,
    multitask_strategy="interrupt",
)

# Wait until the second run completes
await client.runs.join(thread["thread_id"], second_run["run_id"])

# Get the state of the thread
state = await client.threads.get_state(thread["thread_id"])
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

Never mind, Thanksgiving is the 28th! Order Ham for Thanksgiving by next Friday.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_JXHFQRMuRzTMjw3KRs9sR3I0)
 Call ID: call_JXHFQRMuRzTMjw3KRs9sR3I0
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Order Ham for Thanksgiving', 'time_to_complete': 30, 'deadline': '2024-01-05T00:00:00Z', 'solutions': ['Check local grocery stores for ham availability.', 'Consider ordering online for delivery.'], 'status': 'not started'}

Document 45114b64-a6b7-4d24-bbad-b339da309224 updated:
Plan: Update the task to reflect the new deadline and status.
Added content: Send Erik his t-shirt gift this weekend.

Document c3ee571a-321d-4bca-a69a-6f49b273ae54 updated:
Plan: Update the task to reflect the new deadline and s

In [6]:
# Confirm that the first run was interrupted
print((await client.runs.get(thread["thread_id"], interrupted_run["run_id"]))["status"])

interrupted


## Rollback

We can use [rollback](https://langchain-ai.github.io/langgraph/cloud/how-tos/rollback_concurrent/) to interrupt the prior run of the graph and starts a new one with the double-texted input.


In [7]:
# Create a new thread
thread = await client.threads.create()

# Create new ToDos
user_input_1 = "Add a ToDo to call to make appointment at Yoga."
user_input_2 = "Actually, add a ToDo to drop by Yoga in person on Sunday."
config = {"configurable": {"user_id": "Lance"}}
graph_name = "task_maistro" 

rolled_back_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_1)]}, 
    config=config,
)

second_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_2)]}, 
    config=config,
    multitask_strategy="rollback",
)

# Wait until the second run completes
await client.runs.join(thread["thread_id"], second_run["run_id"])

# Get the state of the thread
state = await client.threads.get_state(thread["thread_id"])
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

Actually, add a ToDo to drop by Yoga in person on Sunday.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_Ff9FptNtfXv5bPunXzeOi8jz)
 Call ID: call_Ff9FptNtfXv5bPunXzeOi8jz
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Drop by Yoga in person on Sunday.', 'time_to_complete': 60, 'deadline': '2024-01-07T00:00:00Z', 'solutions': ['Check class schedule for Sunday.', 'Bring yoga mat and water bottle.'], 'status': 'not started'}
================================== Ai Message ==================================

I've added the task to drop by Yoga in person on Sunday to your ToDo list. If you need anything else, just let me know!


In [8]:
# Confirm that the original run was deleted
try:
    await client.runs.get(thread["thread_id"], rolled_back_run["run_id"])
except httpx.HTTPStatusError as _:
    print("Original run was correctly deleted")

Original run was correctly deleted


In [9]:
state = await client.runs.get(thread["thread_id"], rolled_back_run["run_id"])
state


HTTPStatusError: Client error '404 Not Found' for url 'http://localhost:8123/threads/985890f6-6b19-42cd-81be-a86bff91037a/runs/1efc6fa3-d8ab-6c42-a25e-12509d065179'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404